# 🛡️ Interactive Squad Simulation
Follow the directions at the top of each cell. After running the last cell, use the sliders and dropdowns below to run and visualize the simulation.

In [ ]:
# If you are working in colab, use this block to clone the GitHub repo, install dependencies, and pull down the latest changes.
%cd /content/
!rm -rf SE3250-Spring2025-SquadSimulation
!git clone https://github.com/SuprMunchkin/SE3250-Spring2025-SquadSimulation.git
%cd SE3250-Spring2025-SquadSimulation
!pip install -r requirements.txt

!git fetch origin
!git checkout Main #Change this line to use your branch
%ls

In [ ]:
# Import and setup
import sys
import yaml
import matplotlib.pyplot as plt
from ipywidgets import interact

# Custom imports
import os
sys.path.append("../models")
from models.squad_simulation import run_simulation
yaml_path = "config/simulation.yaml"
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

# Define interactive runner
def run_interactive_sim(blue_stock, hostile_stock, direction_deviation, armor_type, environment):
    """
    Run the interactive squad simulation and plot the results.

    Parameters:
        blue_stock (int): Number of blue units.
        hostile_stock (int): Number of hostile units.
        direction_deviation (int): Direction deviation in degrees.
        armor_type (str): Type of armor for blue units.
        environment (str): Simulation environment.
    """
    params = {
        "blue_stock": blue_stock,
        "hostile_stock": hostile_stock,
        "direction_deviation": direction_deviation,
        "armor_type": armor_type,
        "environment": environment
    }
    result = run_simulation(params)

    positions = result['positions']
    hostile_positions = result['hostile_positions']

    plt.figure(figsize=(8, 8))

    # Plot Blue Patrol Path
    if positions:  # Check if there are any blue positions
        x_vals, y_vals = zip(*positions)
        plt.plot(x_vals, y_vals, label='Blue Patrol Path', color='blue')
        plt.scatter(x_vals[0], y_vals[0], c='green', label='Start', zorder=5)
        plt.scatter(x_vals[-1], y_vals[-1], c='purple', label='End', zorder=5)

    # Plot Hostile Position(s)
    if hostile_positions:
        hx_vals, hy_vals = zip(*hostile_positions)
        if len(hostile_positions) == 1:
            # Plot a single dot if there's only one hostile position (stationary)
            plt.scatter(hx_vals[0], hy_vals[0], c='red', label='Hostile Position', zorder=5)
        else:
            # Plot the path if there are multiple hostile positions
            plt.plot(hx_vals, hy_vals, label='Hostile Path', linestyle='--', color='red')

    #plt.xlim(0, 1000)
    #plt.ylim(0, 1000)
    plt.title("Squad Movement Simulation")
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

    print(f"👥 Blue Remaining: {result['blue']['stock']} / {params['blue_stock']}")
    print(f"🔴 Hostile Remaining: {result['hostile']['stock']} / {params['hostile_stock']}")

    # Print statements here can be used for troublshooting.
    #print("Blue positions: ", positions)
    #print("Hostile positions: ", hostile_positions)

# Create interactive widget interface
interact(
    run_interactive_sim,
    blue_stock=(1, 20),
    hostile_stock=(1, 40),
    direction_deviation=(0, 45, 5),
    armor_type=list(config['armor_profiles'].keys()),
    environment=list(config['threat_probs'].keys())
)

In [ ]:
from ipywidgets import Button, VBox, Output
import itertools
import pandas as pd

# Prepare combinations of armor and environment
armor_types = list(config['armor_profiles'].keys())
environments = list(config['threat_probs'].keys())
combinations = list(itertools.product(armor_types, environments))

output = Output()

def run_all_combinations(_):
    output.clear_output()
    blue_stock = 10
    hostile_stock = 40
    direction_deviation = 10

    results = []
    with output:
        for armor, env in combinations:
            params = {
                "blue_stock": blue_stock,
                "hostile_stock": hostile_stock,
                "direction_deviation": direction_deviation,
                "armor_type": armor,
                "environment": env
            }
            result = run_simulation(params)
            blue_remaining = result['blue']['stock']
            hostile_remaining = result['hostile']['stock']
            results.append({
                "Armor": armor,
                "Environment": env,
                "Blue_Remaining": blue_remaining,
                "Hostile_Remaining": hostile_remaining
            })
        # Plotting
        fig, ax = plt.subplots(figsize=(10, 6))
        for armor in armor_types:
            envs = [r['Environment'] for r in results if r['Armor'] == armor]
            blues = [r['Blue_Remaining'] for r in results if r['Armor'] == armor]
            ax.plot(envs, blues, marker='o', label=f'Armor: {armor}')
        ax.set_title("Blue Remaining vs Environment for Each Armor Type")
        ax.set_xlabel("Environment")
        ax.set_ylabel("Blue Remaining")
        ax.legend()
        plt.show()

        # Show results as a table
        df_results = pd.DataFrame(results)
        display(df_results)

run_button = Button(description="Run All Armor/Threat Combinations")
run_button.on_click(run_all_combinations)

VBox([run_button, output])